# Creating a chatbot for structured data using SQL

## Overview
**Generative AI (GenAI)** is a groundbreaking technology that generates human-like texts, images, code, and other forms of content. Although this is all true the focus of many GenAI techniques or implementations have been on unstructured data such as PDF's, text docs, image files, websites, etc. where it is required to set a parameter called *top K*. Top K utilizes an algorithm to only retrieve the top scored pieces of content or docs that is relevant to the users ask. This limits the amount of data the model is presented putting a disadvantage for users that may want to gather information from structured data like CSV and JSON files because they typically want all the occurrences relevant data appears. 

An example would be if you had a table that lists different types of apples, where they originate, and their colors and you want a list of red apples that originate from the US the model would only give you partial amount of the data you need because it is limited to looking for the top relevant data which may be limited to only finding the top 4 or 20 names of apples (depending on how you have configured your model) instead of listing them all. 

The technique that is laid our in this tutorial utilizes **SQL databases** and asks the model to create a query based on the ask of the user. It will then submit that query to the database and present the user with the results. This will not only give us all the information we need but will also decrease the chances of hitting our token limit.

## Prerequisites

We assume you have access to Azure AI Studio, Azure SQL Databases, and have already deployed an LLM. For this tutorial we used **gpt 3.5** and used the **Python 3.10** kernel within our Azure Jupyter notebook.

## Learning objectives

In this tutorial you will learn:
- Setting up a Azure SQL database
- Creating a SQl table and query from it
- Creating a chatbot and utilizing langchains SQL agent to connect the bot to a database

## Table of Contents

- [Summary](#summary)
- [Install Packages](#packages)
- [Create Azure SQL Database](#azure_db)
- [Create Azure SQL Table](#azure_table)
- [Submitting a Query](#query)
- [Setting up a Chatbot](#chatbot)
- [Conclusion](#conclusion)
- [Cleaning up Resources](#cleanup)

## Get started

### Install packages <a id="packages"></a>

In [ ]:
pip install 'pyodbc' 'fast_to_sql' 'sqlalchemy'
pip install --upgrade "langchain-openai" "langchain" "langchain-community"

### Create Azure SQL Database <a id="azure_db"></a>

Follow the instructions [here](https://learn.microsoft.com/en-us/azure/azure-sql/database/single-database-create-quickstart?view=azuresql&tabs=azure-portal) to create a single database in Azure SQL Database. Note that for this tutorials database the field name **Use existing data** was set to **None**.

### Create Azure SQL Table <a id="azure_table"></a>

Now that we have our SQL database we will connect to it using the python package `pyodbc` which will allow us to commit changes to our database and query tables.

In [ ]:
import pyodbc

server_name = "<ENTER_DB_SERVER_NAME>"
user = "<ENTER_DB_USERNAME>"
password = "<ENTER_DB_PASSWORD>"
database = "<ENTER_DB_NAME>"
driver= '{ODBC Driver 18 for SQL Server}'

conn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+server+'.database.windows.net/;PORT=1443;DATABASE='+database+';UID='+user+';PWD='+ password)

Now that we are connected to our database we can upload our data as a table, in this example we are using a csv file from Kaggle that can be downloaded from [here](https://www.kaggle.com/datasets/henryshan/2023-data-scientists-salary). 

**Tip:** If you are using a json file you can used the command `pd.read_json` to load in the data frame.

In [ ]:
import pandas as pd
import numpy as np 
# reading the csv file using read_csv and storing the data frame in variable called df
df = pd.read_csv('ds_salaries.csv')

# view the data
df.head()

**Tip:** If you receive a **timeout error** wait a couple of minutes and then run the above code again.

Our second python package we are using is `fast_to_sql` **(fts)** which will allow us to easily create tables from our data. Usually, you would have to create a SQL query that outlines the columns, datatype, and values of our table but **fts** does all the work for us.

In [ ]:
from fast_to_sql import fast_to_sql as fts
table_name = "ds_salaries"
create_table = fts(df, table_name , conn, if_exists="replace", temp="FALSE")

Now we will commit our change to make it permanent.

In [ ]:
conn.commit()

### Submiting a query <a id="query"></a>

To submit a query to our database we first need to establish our connection with a **cursor** which allows you to process data row by row.

**Tip:** At any time you can close the connection to your database using the command `conn.close()`.

In [ ]:
cursor = conn.cursor()

Now we can finally submit a query to our database! In the query below we ask to count the number of workers that worked in 2023. Then we use the `execute` command to send our query to the database. The result will be an **iterable** which we will need to create a for loop to see our query result. the result you should receive is **1785**.

In [ ]:
query="SELECT COUNT(work_year) FROM ds_salaries WHERE work_year = '2023';"

cursor.execute(query)
for row in cursor:
    print(f'QUERY RESULT: {str(row)}') 

Another way to output our query is to make it into a list and we can use the python function `replace` to get rid of the parentheses.

In [ ]:
query="""SELECT name FROM sys.columns WHERE object_id = OBJECT_ID('ds_salaries')  
"""
cursor.execute(query)

result = [str(row).replace("('", "").replace("',)", "") for row in cursor]

print(result)

### Setting up a chatbot <a id="chatbot"></a>

For our chatbot we will be utilizing langchain to connect our model to our database.

In [ ]:
#load in the required tools
from langchain_openai import AzureChatOpenAI
from sqlalchemy import create_engine
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

Enter in your OpenAI model's endpoint and key. For this tutorial we used gpt 3.5.

In [ ]:
import os
os.environ["AZURE_OPENAI_ENDPOINT"] = "<ENTER_ENDPOINT>"
os.environ["AZURE_OPENAI_API_KEY"] = "<ENTER_KEY>"

Set our model to the variable `llm` and enter the model name which was set when the model was deployed, this will connect langchain to our model. We are also setting the **temperature** to **0** because we don't want any randomness or creativity in the models answer only what is in the date.

In [ ]:
model_name="<Enter_Model_Name>"

llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment=model_name,
    temperature = 0
)

The first step to connecting our model to our database will be to create an engine that will help langchain connect to our SQL database using a package called `sqlalchemy`. The package will take the same info from the connection we name before but the format of driver is a little different where in this package it does not require curly brackets.

In [ ]:
driver= "ODBC Driver 18 for SQL Server"

The database information will be entered as a connection string and then converted to our database engine using the command `create_engine`.

In [ ]:
py_connectionString=f"mssql+pyodbc://{user}:{password}@{server_name}.database.windows.net/{database}?driver={driver}"
db_engine = create_engine(py_connectionString)

Now that we have established a connection to the databse we need to use the langchain package `SQLDatabase` to pass that connection to langchain. Notice that we leave the schema as **"dbo"** which stands for database owner and will be the default schema for all users, unless some other schema is specified. The dbo schema cannot be dropped.

In [ ]:
db = SQLDatabase(db_engine, view_support=True, schema="dbo")

Lets run a test query below to ensure we are connected!

In [ ]:
print(db.dialect)
db.run("SELECT COUNT(*) FROM ds_salaries WHERE work_year = 2023 AND experience_level = 'SE' ")

The last step will be to create a SQL agent. The SQL agent will provide our bot with the following instructions:
1. Taken in the users ask or question and survey the SQL table mentioned in the ask/question
2. Create a SQL query based on the columns that have relevant information to the ask/question
3. Submit the query to our database and present the results to the user

There is no need for a prompt because the agent already supplies that.

**Tip**: If you do not want to see the reasoning of the agent and only want to answer set `verbose` to `false` (e.g., `verbose=False`)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(llm=llm,
toolkit=toolkit,
verbose=True,
agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

Now we can ask our bot questions about our data! Notice how in the question below we mention that the table we are looking at is **ds_salaries**.

In [ ]:
question = "count the number of employees that worked in 2023 and have a experience level of SE in table ds_salaries."

In [ ]:
agent_executor.invoke(question)

## Conclusion <a id="conclusion"></a>

In this notebook you learned how to set up a Azure SQL database and connect your model to the database using langchain tools, creating a chatbot that can read and retrieve data from structured data formats.

## Clean up <a id="cleanup"></a>

Dont forget to turn off or delete any notebooks or compute resources! Below you will find instructions to delete the SQL database. With the first step to close the connection to the database.

In [ ]:
conn.close()

We will be using Azure CLI commands which first require use to login. Run the command below and follow the steps outputted.

In [ ]:
! az login

 Next we will delete our database, wait for the command to output **'Finished'**.

In [ ]:
resource_group="<Enter_Resource_Group>"
!az sql db delete --name {database} --resource-group <Enter_ResourceGroup> --server {server_name}

For this command you will need your subscriptions ID which can be found running the following command:

In [ ]:
!az sql server list --resource-group {resource_group}

Finally delete your SQL server, wait for the command to output **'Finished'**.

In [ ]:
subscription_id='<Enter_Subscription_ID>'
!az sql server delete --name {server_name} --resource-group {resource_group} --subscription {subscription_id}  -y